# LIGO-Virgo EM Follow-Up Tutorial

by Leo P. Singer (NASA/GSFC) <leo.singer@ligo.org>

This document is [https://dcc.ligo.org/G1500442](LIGO-G1500442-v1).


## Abstract

This document explains how to receive, filter, and process gravitational-wave (GW) detection candidate alerts from Advanced LIGO and Virgo. We provide sample code in Python and document alternatives for users of other programming environments. You can download this document and run the code samples in [IPython Notebook](http://ipython.org/notebook.html).

## Introduction

The LIGO-Virgo data are analyzed in near real-time to search for GW transients due to compact binary coalescence (CBC) events or unmodeled "burst" sources. For each detection candidate, a series of alerts are produced and distributed via the [Gamma-ray Coordinates Network (GCN)](http://gcn.gsfc.nasa.gov).

The GCN system may already be familiar to some, as it is has been in use since the early 1990s to transmit times and coordinates of gamma-ray bursts (GRBs) detected by gamma-ray space missions to ground-based observers. However, there are three peculiariaties of GW observations that veteran GCN users should be aware of:

1. Unlike GRBs, GW sources cannot (always) be localized to a unique sky location; GW position reconstructions are (sometimes) multimodal and non-Gaussian. As a consequence, LIGO-Virgo GCN notices do *not* contain a RA, Dec, or error radius; instead they contain a pointer URL to a FITS file containing a probability sky map in the [HEALPix](http://healpix.sourceforge.net) all-sky projection.
1. GCN notices will produce LIGO-Virgo candidates that span a range of significances. The significance of a candidate is reported as the false alarm rate (FAR). The FAR of a detection candidate measures approximately how frequently an event of similar strenght occurs due to chance noise fluctuations or instrumental glitches. A FAR of 1/century ($\sim 3 \times 10^{-10}$ Hz) is generally regarded as sufficient for a "confident" detection. However, alerts will produced for (**FIXME**) all events with FAR≥1/month ($\sim 4\times10^{-7}$ Hz), such that there should be at least one alert per month on average.
1. The first few GW alerts will be distributed privately to LIGO MOU partners, over a purpose-built GCN connection. (**FIXME: link to description of policy**)

## 1. Sign up for GCN network

The first step is to sign up for the GCN network. You will need a computer with a static IP address and you will need to register the IP address and port number from which you will connect to the GCN network. (Note: You can have multiple GCN subscriptions on different IP addresses/ports).)

There is a [standard signup process for GCN](http://gcn.gsfc.nasa.gov/invitation.html), but only the standard notice types are supported. To receive the LIGO-Virgo GCNs, you must contact Scott Barthelmey (NASA/GSFC) directly. Fill in the example e-mail below with the name of your site, your contact information, and the IP address/port on which you want to receive GCNs.

    To: scott@lheamail.gsfc.nasa.gov
    Subject: LSC/Virgo GCN subscription

    Dear Scott,

    Please sign me up for receiving all LIGO-Virgo GCN notices
    (LVC_PRELIMINARY, LVC_INITIAL, and LVC_UPDATE). See below
    for site and connection details.

    Best,
    Edwin Hubble

    Site: Mount Wilson **FIXME**
    Contact: Edwin Hubble <hubble@obs.carnegiescience.edu> **FIXME**
    IP Address: 22.231.113.64:8096 **FIXME**
    Distribution Method: VOEvent/XML (all servers)

## 2. Sign up for a download token

You will need a *robot certificate* to download files that are linked from the GCN notices.

Follow the instructions at **FIXME** https://wiki.ligo.org/AuthProject/LIGOCARobotCertificate to request and obtain a robot certificate. Save the resulting `robot.cert` file to the machine on which you have elected to receive GCNs.

## 3. Install some dependencies

You will need to install a few third-party Python packages to run the example code in this tutorial. These include:

* [pygcn](https://pypi.python.org/pypi/pygcn) for connecting to GCN (alternatives: [comet](https://pypi.python.org/pypi/Comet))
* [requests](https://pypi.python.org/pypi/requests) for easy HTTP downloads in Python (many [alternatives](https://docs.python.org/2/library/urllib.html) in Python standard library)
* [healpy](https://pypi.python.org/pypi/healpy), for decoding HEALPix images (alternatives: [DS9](http://ds9.si.edu), [Aladin](http://aladin.u-strasbg.fr), [HEALPix bindings for C/C++/Fortran/Java/IDL](http://healpix.sourceforge.net))
* [astropy](https://pypi.python.org/pypi/astropy) version 1.0.1 or newer (optional, for computing observability windows, etc.)

If you are on a Mac and use the [MacPorts](http://macports.org) package manager, you can install all of the above with the following command:

    $ sudo port install py27-gcn py27-lxml py27-requests py27-healpy py27-astropy

Otherwise, the fastest way to install the dependencies is with [`pip`](https://pip.pypa.io/en/latest/quickstart.html), a package manager that comes with most Python distributions. To install these packages with `pip`, run the following command:

    $ pip install pygcn lxml requests healpy astropy

## 4. Write GCN handler script

Now we'll write a GCN handler script. First, some imports...

In [ ]:
import gcn
import gcn.handlers
import gcn.notice_types
import requests
import healpy as hp

Next, we'll write a function that we want to get called every time a GCN is received. We will use the `@gcn.handlers.include_notice_types` [https://docs.python.org/3/glossary.html#term-decorator](function decorator) to specify that we only want to process certain notice types. There are three notice types:

1. `LVC_PRELIMINARY`: Provides the time, significance, and basic parameters about a GW detection candidate. No localization information. Sent with a latency of a minute or so.
1. `LVC_INITIAL`: A rapid sky localization is available. Sent with a latency of a few minutes.
1. `LVC_UDPATE`: A refined sky localization is availaable. Sent with a latency of hours or more.

In the following example, we will process only the last two types (`LVC_INITIAL` and `LVC_UPDATE`), which contain links to sky map FITS files. The following handler function will parse out the URL of the FITS file, download it, and extract the probability sky map.

**FIXME**: Untested, because the GCN VOEvents are currently malformed as XML documents (missing a closing tag).

In [ ]:
# Function to call every time a GCN is received.
# Run only for notices of type LVC_INITIAL or LVC_UPDATE.
@gcn.handlers.include_notice_types(
    gcn.notice_types.LVC_INITIAL,
    gcn.notice_types.LVC_UPDATE)
def process_gcn(payload, root):
    # Read out integer notice type
    notice_type = int(root.find("./What/Param[@name='Packet_Type']").attrib['value'])

    # Read out URL of sky map.
    # This will be something like
    # https://gracedb.ligo.org/api/events/M131141/files/bayestar.fits.gz
    skymap_url = root.find("./What/Group[@type='GW_SKYMAP']/Param[@name='skymap_fits_x509']").attrib['value']

    # Retrieve sky map (goes into in-memory file-like object)
    # FIXME: need to provide cert here!
    response = requests.get(skymap_url, stream=True)

    # Raise an exception unless the download succeeded (HTTP 200 OK)
    response.raise_for_status()

    # Read HEALPix data from FITS stream
    # (Note: reading FITS files from arbitrary file-like objects
    # requires Astropy or a very recent version of pyfits; on
    # older systems you may have to save the download to a temporary
    # file.)
    skymap = hp.read_map(response.raw)

Finally, we will listen for GCNs. You need to tell the `gcn.listen` function on what port you want to connect to the GCN network; this should be the same port that you provided to Scott Barthelmey in Section 1 above. You also need to tell `gcn.listen` which function to call whenever it receives an GCN; this will be the `process_gcn` function that we just defined.

When you run the following code snippet, the `gcn.listen` function will continue until you interrupt the program (by pressing the stop button in IPython Notebook, typing Control-C in the terminal, or sending a kill signal to your Python script).

Note: `gcn.listen` will automatically reconnect to the GCN network if the network connection is broken.

In [ ]:
# Listen for GCNs until the program is interrupted
# (killed or interrupted with control-C).
gcn.listen(port=8096, handler=process_gcn)